# ECON 326: Issues in Regression using R

## Authors
* Jonathan Graves (jonathan.graves@ubc.ca)
* Devan Rawlings (rawling5@student.ubc.ca)

## Prerequisites
* Multiple regression
* Simple regression
* Data analysis and introduction

## Outcomes

* Understand the origin and meaning of multicollinearity in regression models
* Perform simple tests for multicollinerarity using VIF
* Be able to demonstrate common methods to fix or resolve collinear data
* Understand the origin and meaning of heteroskedasticity in regression models
* Perform a variety of tests for heteroskedasticity
* Compute robust standard errors for regression models
* Understand other techniques for resolving heteroskedasticity in regression models

### Notes

<span id="fn1">[<sup>1</sup>](#fn1s)Data is provided under the Statistics Canada Open License.  Adapted from Statistics Canada, 2016 Census Public Use Microdata File (PUMF). Individuals File, 2020-08-29. This does not constitute an endorsement by Statistics Canada of this product.</span>

<span id="fn2">[<sup>2</sup>](#fn2s)Stargazer package is due to: Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer </span>


In [ ]:
#load the data and set it up

library(tidyverse)
library(haven)
library(car)
#library(sandwich)
library(lmtest)

#install.packages("stargazer")  #un-comment this and run this line if you don't have stargazer installed
library(stargazer)
source("hands_on_tests_4.r")
source("sandwich.r")

census_data <- read_dta("02_census2016.dta")
census_data <- as_factor(census_data)
census_data <- census_data %>%
               mutate(lnwages = log(wages))
census_data <- filter(census_data, !is.na(census_data$wages))
census_data <- filter(census_data, !is.na(census_data$mrkinc))
glimpse(census_data)

In this week's exercise, we will explore several important issues in multiple regression models, and explore how to test, evaluate, and correct them where appropriate.  It is important to remember that there can be many other issues that arise in specific models; as you learn more about econometrics and create your own research questions, different issues will arise.  Consider these as "examples" for some of the most common issues that arise in regression models.

# Part 1: Multicollinearity

Multi-collinearity is a surprisingly common issue in applied regression analysis.  It refers to the situation where a variable is "overdetermined" by the other variables in a model - okay, that's not really much clearer.  Let's look at the problem mathematically; in calculating an OLS estimation, you are estimating a relationship like:

$$Y_i = \beta_0 + \beta_1 X_i + \epsilon_i$$

You find the estimates of the coefficients in this model using OLS; i.e. solving an equation like:

$$ \min_{b_0, b_1} \sum_{i=1}^n(Y_i - b_0 -b_1 X_i)^2 $$

Under the OLS regression assumptions, this has a unique solution; i.e you can find unique values for $b_0$ and $b_1$.  

However, what if you wrote an equation like this:

$$Y_i = \beta_0 + \beta_1 + \beta_2 X_i + \epsilon_i$$

This _seems_ like it would be fine, but remember what you are doing: trying to find a _line_ of best fit.  The problem is that this equation does not define a unique line; the "intercept" is $\beta_0 + \beta_1$.  There are two "parameters" ($\beta_0, \beta_1$) for a single "characteristics" (the intercept).  This means that the resulting OLS problem:

$$ \min_{b_0, b_1, b_2} \sum_{i=1}^n(Y_i - b_0 -b_1 -b_2 X_i)^2 $$

Does not have a unique solution.  In geometric terms, it means you can find many representations of a line with two intercept parameters.  This is referred to in econometrics as a lack of **identification**; multicollinearity is one way that identification can fail in regression models.

You can see this in the following example, which fits an OLS estimate of ``mrkinc`` and ``wages`` then compares the fit to a regression with two intercepts.  Try changing the values to see what happens!

In [ ]:
reg <- lm(wages ~ mrkinc, data = census_data)

print("Regression Coefficients and SR:")
round(reg$coef,2)

b_0 <- reg$coef[[1]]
b_1 <- reg$coef[[2]]

resid <- census_data$wages - b_0 - b_1*census_data$mrkinc

round(sum(resid),10)

print("Two Intercept Coefficients and SR")

k <- 5000 #change me!

b_0 = reg$coef[[1]]/2 - k
b_1 = reg$coef[[1]]/2 + k
b_2 = reg$coef[[2]]

resid <- census_data$wages - b_0 - b_1 - b_2*census_data$mrkinc

round(sum(resid),10)



Make sure you understand what the example above is doing.  Notice how the results are exactly the same, no matter what the value of ``k`` is?

Okay, you're probably thinking, that makes sense - but just don't write down an equation like that!  After all, it seems somewhat artificial that we added an extra intercept term... which is true!

However, multicollinearly can occur with _any_ set of variables in the model; not just the intercept.  For example, suppose you have a multiple regression:

$$Y_ i = \beta_0 + \beta_1 X_{1,i} + \beta_2  X_{2,i} + \beta_3  X_{3,i} + \epsilon_i$$

What would happen if there was a relationship between $X_1, X_2$ and $X_3$ like:

$$X_{1,i} = 0.4 X_{2,i} + 12 X_{3,i}$$  

When, we could then re-write the equation as:

$$Y_ i = \beta_0 + \beta_1 (0.4 X_{2,i} + 12 X_{3,i}) + \beta_2  X_{2,i} + \beta_3  X_{3,i} + \epsilon_i$$

$$\implies Y_ i = \beta_0 + (\beta_2 + 0.4 \beta_1)  X_{2,i} + (\beta_3 + 12 \beta_1)X_{3,i} + \epsilon_i$$

Oh no!  The same problem is now occuring, but with $X_2$ and $X_3$: the slope coefficients depend on a free parameter ($\beta_1$).  You cannot uniquely find the equation of a line (c.f. plane) with this kind of equation.

You can also intuitively see the condition here: multicollinearity occurs when you can express one variable as a _linear_ combination of the other variables in the model.

* This is sometimes referred to as **perfect multicollinearity**, since the variable is _perfectly_ expressed as a linear combination of the other variable.
* The linearity is important because this is a linear model; you can have similar issues in other models, but it has a special name in linear regression

## Perfectly Multicollinearity in Models

In general, most statistical packages (like R) will automatically detect, warn, and remove perfectly multicollinear variables from a model; this is because the algorithm they use to solve problems like the OLS estimation equation detects the problem and avoids a "crash".  This is fine, from a mathematical perspective - since mathematically the two results are the same (in a well-defined sense, as we saw above).

However, from an economic perspective this is very bad - it indicates that there was a problem with the _model_ that you defined in the first place.  Usually, this means one of three things:

1. You included a set of variables which were, in combination, identical.  For example, including "family size" and then "number of children" and "number of adults" in a regression
2. You did not understand the data well enough, and variables had less variation than you thought they did - conditional on the other variables in the model.  For example, maybe you thought people in the dataset could have both graduate and undergraduate degrees - so there was variation in "higher than high-school" but that wasn't true
3. You wrote down a model which was poorly defined in terms of the variables.  For example, you including all levels of a dummy variable, or included the same variable measured in two different units (wages in dollars and wages in 1000s of dollars).

In all of these cases, you need to go back to your original regression model and re-evaluate what you are trying to do in order to simplify the model or correct the error.  

Consider the following regression model, in which we want to study whether or not there is a penalty for being a young adult working in Canada:

In [ ]:
census_data <- census_data %>%
    mutate(ya = case_when(
        agegrp == "18 to 19 years" ~ "Yes",
        agegrp == "20 to 24 years" ~ "Yes",
        agegrp == "25 to 29 years" ~ "Yes",
        TRUE ~ "No" #this is for all other cases
    )) %>%
    mutate(ya = as_factor(ya))

regression2 <- lm(wages ~ immstat + ya + agegrp, data = census_data)

summary(regression2)

Can you see why the multi-collinearity is occuring here?  Try to write down an equation which points out what the problem is in this regression - why is it multicollinear?  How could you fix this problem by changing the model?

> _Think Deeper_: You will notice, above, that it excluded the "25 to 29 years" agegroup.  Did it have to exclude that one?  Could it have excluded another one instead?  What do you think?

## Imperfect Multicollinearity

A related issue to perfect multicollinearity is "near" (or **imperfect**) multicollinearity.  If you recall from the above, perfect multicollinearity occurs when you have a relationship like:

$$X_{1,i} = 0.4 X_{2,i} + 12 X_{3,i}$$  

Notice that in this relationship it holds _for all_ values of $i$.  However, what if it held for _nearly_ all $i$ instead?  In that case, we would still have a solution to the equation... but there would be a problem.  Let's look at this in the simple regression case.  


$$Y_i = \beta_0 + \beta_1 X_i + \epsilon_i$$

Now, let's suppose that $X_i$ is "almost" collinear with $\beta_0$.  To be precise, suppose that $X_i = 15$ for $k$-% of the data ($k$ will be large) and $X_i = 20$ for $(1-k)$-% of the data.  This is _almost_ constant, and so it is _almost_ collinear with $\beta_0$ (the constant).  Let's also make the values of $Y_i$ so that $Y_i(X_i) = X_i + \epsilon_i$ (so $\beta_1 = 1$), and we will set $\sigma_Y = 1$

This implies that (applying some of the formulas from class):

$$\beta_1 = \frac{Cov(X_i,Y_i)}{Var(X_i)} = 1$$

$$s_b = \frac{1}{\sqrt{n-2}}\sqrt{\frac{1}{r^2}-1}$$

$$r = \frac{\sigma_X}{\sigma_Y}$$

As you can see, when $Var(X_i)$ goes down, $\sigma_X$ falls, and the value of $r$ falls; intuitively, when $k$ rises, the variance will go to zero, which makes $r$ go to zero as well (since there's no variation).  You can then see that $s_b$ diverges to infinity.  

We can make this more precise.  In this model, how does $Var(X_i)$ depend on $k$?  Well, first notice that $\bar{X_i} = 15\cdot k + 20 \cdot (1-k)$.  Then,

$$Var(X_i) = (X_i - \bar{X_i})^2 = k (15 - \bar{X_i})^2 + (1-k)(20 - \bar{X_i})^2$$

$$\implies Var(X_i) = 25[k(1-k)^2 + (1-k)k^2]$$

Okay, that looks awful - so let's plot a graph of $s_b$ versus $k$ (when $n = 1000$):

In [ ]:
options(repr.plot.width=6,repr.plot.height=4)

r = 0.01 #change this to see how the granularity matters!

eq = function(k){(1/sqrt(1000-2))*(1/(25*(k*(1-k)^2 + (1-k)*k^2))-1)}
s = seq(0.5, 1.00, by = r)
n = length(s)

plot(eq(s), type='l',  xlab="Values of K", ylab="Standard Error", xaxt = "n")
axis(1, at=seq(0, n-1, by = 10), labels=seq(0.5, 1.00, by = 10*r))

# You will notice that the plot actually diverges to infinity
# Try making R smaller to show this fact!
#Notice the value at 1 increases

Why does this happen?  The reason actually has to do with _information_.

When you estimate a regression, you are using the variation in the data to estimate each of the parameters.  As the variation falls, the estimation gets less and less precise, because you are using less and less data to make an evaluation.  The magnitude of this problem can be quantified using the **VIF** or **variance inflation factor** for each of the variables in question.

We can calculate this directly in R by using the ``vif`` function.  Let's look at the collinearity in our model:

In [ ]:
regression2 <- lm(wages ~ immstat + pr + fol, data = census_data)

summary(regression2)

vif(regression2)

As you can see, this model is not particularly collinear.  However, what if we did something like this: regress ``wages`` on ``mrkinc`` plus another variable (``temp``) which is closely related to ``mrkinc``:

In [ ]:
k <- 1000 #change me to see how the results change

temp <- census_data$mrkinc + runif(nrow(census_data), 0, k)
census_data$temp <- temp

regression2 <- lm(wages ~ mrkinc + temp, data = census_data)

summary(regression2)

vif(regression2)

Notice the extremely large VIF.  This would indicate that you have a problem with collinearity in your data.

> _Think Deeper_:  What happens to the VIF as ``k`` changes?  Why?  Can you explain?

There are no "hard" rules for what makes a VIF too large - you should think about your model holistically, and use it as a way to investigate whether you have any problems with your model evaluation and analysis.

# Part 2: Heteroskedasticity

That's a 5-cent word for sure.  **Heteroskedasticity** (Het-er-o-sked-as-ti-city) is a common problem in many economic models.  It refers to the situation in which the distribution of the residuals changes as the explantory variables change.  For example, consider this regression:



In [ ]:
regression3 <- lm(wages ~ mrkinc, data = census_data)

ggplot(data = census_data, aes(x = mrkinc, y = regression3$residuals)) + geom_point() + labs(x = "Market Income", y = "Residuals")

This obviously does not look like a distribution which is unchanging as market income changes.  This is a good "eyeball test" for heteroskedasticty.  Why does heteroskedasticity arise?  For many reasons:

1.  It can be a property of the data; it just happens that some values show more variation, due to the process which creates the data.  One of the most common ways this can arise is where there are several different economic processes creating the data.  For example, in the above, they might be one process for "typical" Canadians and one for "wealthy" Canadians.  The model has trouble fitting both of them.

2.  It can be because of an unobserved variable.  This is similar to above; if we can quantify that process in a variable or a description, we have left it out.  This could create bias in our model, but it will also show up in the standard errors in this way.
3.  It can be because of your model specification.  Models, by their very nature, can be heteroskedastic (or not); we will explore one important example later in this worksheet.
4.  There are many other reasons, which we won't get into here.

Whatever the reason it exists, you need to correct for it - if you don't, while your coefficients will be OK, your standard errors will be incorrect.  You can do this in a few ways.  The first way is to try to change your variables that the "transformed" model (a) makes economic sense, and (b) no longer suffers from heteroskedasticity.  For example, perhaps a _log-log_ style model might work here:


In [ ]:
census_data <- census_data %>%
               mutate(lnmrkinc = log(mrkinc))

temp_data <- filter(census_data, !is.na(census_data$lnmrkinc))
temp_data <- filter(temp_data, !is.na(temp_data$lnwages))
#getting rid of NAs

regression4 <- lm(lnwages ~ lnmrkinc, data = temp_data)

ggplot(data = temp_data, aes(x = lnmrkinc, y = regression4$residuals)) + geom_point() + labs(x = "Log of Market Income", y = "Residuals")

As you can see, that didn't really work.  This is pretty typical: when you transform a model by changing the variables, what you are really doing is adjusting how you think the data process should be described so that it's no longer heteroskedastic.  If you aren't correct with this, you won't fix the problem.

For example, in a _log-log_ model, we are saying "there's a multiplicative relationship"... but that probably doesn't make sense here.  This is one of the reasons why data transformations are not usually a good way to fix this problem unless you have a very clear idea of what the transformation _should_ be.

The most robust (no pun intended) way is to simply use standard errors which are robust to heteroskedasticity.  There are actually a number of different versions of these (which you don't need to know about), but they are all called **HC** or **hetereoskedasticity-corrected** standard errors.  In economics, we typically adopt White's versions of these (called **HC1** in the literature); these are often referred to in economics papers as "robust" standard errors (for short).

This is relatively easy to do in R.  Basically, you run your model, as normal, and then re-test the coefficients to get the correct error using the ``coeftest`` command, but specifying which kind of errors you want to use.  Here is an example:

In [ ]:
regression5 <- lm(wages ~ mrkinc, data = census_data)

summary(regression5)

coeftest(regression5, vcov = vcovHC(regression5, type = "HC1"))

As you can see, the standard errors (and significance tests) give different results; in particular, the HC1 errors are almost 10-times larger than the uncorrected errors.  In this particular model, it didn't make much of a different to the conclusions (even though it changed the $t$ statistics a lot), but it can sometimes change your results.

## Testing for Heteroskedasticity

You can also perform some formal tests for heteroskedasticity.  We learned about two of them in class:

1.  White's Test, which relies on performing a regression using the residuals
2.  Breusch-Pagan Test, which also relies on performing a simpler regression using the residuals

Both of them are, conceptually, very similar.  Let's try (2) for the above regression:

In [ ]:
regression2 <- lm(wages ~ mrkinc, data = census_data)

census_data$resid_sq <- (regression2$residuals)^2

regression3 <- lm(resid_sq ~ mrkinc, data = census_data)

summary(regression3)

Inspecting the results, we can see from the $F$-statistic that we can strongly reject the assumption of homoskedasticity.  This data looks like it's heteroskedastic, because the residuals can be predicted using the explanatory variables.

There is one very important note: 

* If you **fail** one of these tests, it implies that your data is heteroskedastic
* If you **pass** one of these tests, it _does not_ imply that your data is homoskedastic (i.e. not heteroskedastic)

This is because these are statistical tests, and the null hypothesis is "not heteroskedastic".  Failing to reject the null does not mean that the null hypothesis is correct - it just means that you can't rule it out.  This is one of the reasons many economists recommend that you _always_ use robust standard errors unless you have a really compelling reason to believe otherwise.

## Linear Probability Models

How can a model naturally have heteroskedastic standard errors?  It turns out that many common, and important, models have this issue.  In particular, the **linear probability** model has this problem.  If you recall, a linear probability model is a linear regression in which the dependent variable is a dummy.  For example:

$$D_i = \beta_0 + \beta_1 X_{1,i} + \beta_2 X_{2,i} + \epsilon_i$$

These model are very useful because the coefficients have the interpretation as being the change in the probability of the dummy condition occuring.  For example, we previously regressed ``immstat`` in these models to investigate who was more (or less) likely to be an immigrant (or not).  However, this can easily cause a problem when estimated using OLS - the value of $D_i$ must be 0 or 1, and the fitted values (which are probabilities) must be between 0 and 1.

However, _nothing_ in the OLS model forces this to be true.  If you estimate a value for $\beta_1$, if you have an $X_{1,i}$ that is high or low enough, the fitted values will be above or below 1 or 0 (respectively).  This implies that _mechanically_ you have heteroskedasticity because high or low values of the explanatory variables will ALWAYS fit worse than intermediate values.  For example, let's look at the fitted values from this regression:

In [ ]:
census_data$dummy_immstat = as.numeric(census_data$immstat) - 1 #what is this line of code doing?  

regression6 <- lm(dummy_immstat ~ mrkinc, data = census_data)

census_data$fitted <- predict(regression6, census_data)

summary(regression6)

ggplot(data = census_data, aes(x = mrkinc, y = fitted)) + geom_point() + labs(x = "Market Income", y = "Predicted Probability")

Notice how that as $y$ gets larger the fitted value drops.  If someone has a market income of over 2.5 million dollars, they would be predicted to have a negative probability of being an immigrant - which is impossible.

This is why you _always_ must use robust standard errors in these models - even if a test says otherwise!



# Part 3: Exercises

In these exercises, you will get some hands-on experience with testing and correcting for heteroskedasticity and multicollinearity. You will also start to think about the mechanics and uses of non-linear regressions.

### Practical Activity 1

Multicollinearity may seem to be an abstract concept, so let's explore this issue with a practical example. 

Suppose that we are looking to explore the relationship between a worker's income and their immigrant status. For instance, we want to know whether workers with higher incomes in Canada are more likely to be immigrants (i.e., do a lot of high-skill immigrants come to Canada and realize high incomes?). Recall that we have two measures of income: ``wages`` and ``mrkinc``. Both measures of income are informative: ``wages`` refers to gross annual income (before taxes) that employers pay to employees; ``mrkinc`` (market income) refers to net income after government transfers and taxes have been deducted. 

Since they are both good measures of income, we decide to put them both in our regression:

$$I_i = \beta_0 + \beta_1 W_i + \beta_2 M_i + \epsilon_i$$

$I_i$ denotes the dummy variable for whether the person is an immigrant, $W_i$ denotes wages, and $M_i$ denotes market income.

#### Short Answer 1

<em>Prompt:</em> What concern should we have about this regression equation? Explain your intuition.

<font color="red">Answer here in red</font>

Before we continue, let's reduce the sample size of our data set to 200 observations. We will also revert ``immstat`` into a numeric variable:

In [ ]:
#Run this!
census_data200 <- head(census_data, 200) %>%
    mutate(immstat = as.numeric(immstat))

Run the regression described above.

<em>Tested Objects:</em> ``reg1``.

In [ ]:
#Quiz 1

reg1 <- lm(????, data = census_data200)

summary(reg1)

test_1() #test1

#### Short Answer 2

<em>Prompt:</em> What do you notice about the characteristics of the estimated regression? Does anything point to your concern being valid?

<font color="red">Answer here in red</font>

Now, let's suppose we drop 50 more observations:

In [ ]:
#Run this!
census_data150 <- head(census_data200, 150)

Run the regression model again and compare it with the previous regression.

<em>Tested Objects:</em> ``reg2``.

In [ ]:
#Quiz 2

reg2 <- lm(???, data = census_data150)

summary(reg2)

test_2() 

#### Short Answer 3

<em>Prompt:</em> What happened to the regression estimates when we dropped 50 observations? Does this point to your concern being valid?

<font color="red">Answer here in red</font>

Next, increase the sample size back to its full size and run the regression once again.

<em>Tested Objects:</em> ``reg3``

In [ ]:
#Quiz 3

census_data <- as_factor(census_data) %>%
    mutate(immstat = as.numeric(immstat))

reg3 <- lm(???, data = census_data) #fill me in

summary(reg3)

test_3() #test2

#### Short Answer 4

<em>Prompt:</em> What happens to the regression estimates when we increase the sample size back to its full size? Has this helped to address the concern that you had earlier? Explain.

<font color="red">Answer here in red</font>

Is there something else that we can do to eliminate our concern about the regression? Run the regression again with this change.

<em>Tested Objects:</em> ``reg4``.

In [ ]:
#Quiz 4

reg4 <- lm(???, data = census_data) #use only wages

summary(reg4)

test_4()

#### Short Answer 5

<em>Prompt:</em> Did this change eliminate the concern? How do you know?

<font color="red">Answer here in red</font>

### Practical Activity 2

Heteroskedasticity is another issue that researchers frequently deal with when they estimate regression models. Consider the following regression model:

$$W_i = \alpha_0 + \alpha_1 S_i + \alpha_2 D_i + \alpha_3 I_i + \epsilon_i$$

$W_i$ denotes wages, $S_i$ is level of schooling, $D_i$ is a dummy variable for being female and $I_i$ is a dummy variable for being an immigrant.

#### Short Answer 6
<em>Prompt:</em> Should we concerned about heteroskedasticity in this model? If so, what is the potential source of heteroskedasticity, and what do we suspect to be the relationship between the regressor and the error term?

<font color="red">Answer here in red</font>

#### Short Answer 7

<em>Prompt:</em> If we suppose that heteroskedasticity is a problem in this regression, what consequences will this have for our regression estimates?

<font color="red">Answer here in red</font>

Before we proceed, let's refine ``hdgree`` in the same way that we have in previous worksheets by running the code below:

In [ ]:
#Run this!
census_data <- 
        census_data %>%
        mutate(educ = case_when(
              hdgree == "no certificate, diploma or degree" ~ "Less than high school",
              hdgree == "secondary (high) school diploma or equivalency certificate" ~ "High school diploma",
              hdgree == "trades certificate or diploma other than certificate of apprenticeship or certificate of qualification" ~ "Some college",
              hdgree == "certificate of apprenticeship or certificate of qualification" ~ "Some college",
              hdgree == "program of 3 months to less than 1 year (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "program of 1 to 2 years (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "program of more than 2 years (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "university certificate or diploma below bachelor level" ~ "Some college",
              hdgree == "bachelor's degree" ~ "Bachelor's degree",              
              hdgree == "university certificate or diploma above bachelor level" ~ "Graduate school",
              hdgree == "degree in medicine, dentistry, veterinary medicine or optometry" ~ "Graduate school",
              hdgree == "master's degree" ~ "Graduate school",
              hdgree == "earned doctorate" ~ "Graduate school",
              hdgree == "not available" ~ "not available"
              )) %>%
        mutate(educ = as_factor(educ)) %>%
        mutate(educ = factor(educ, levels=c("not available", "Less than high school", "Some college", "Bachelor's degree", "Graduate school"))) #Reorders the factor levels for presentation purposes

Run the regression above, and graph the residuals against the level of schooling.

<em>Tested Objects:</em> ``reg5``. The graph will be graded manually.

In [ ]:
#Quiz 5
census_data <- na.omit(census_data)

#Run the regression
reg5 <- lm(???, data = census_data)

resiplot <- ggplot(reg5, aes(x = educ, y = .resid)) + xlab("Education Level") + ylab("Wages (Residuals)")
resiplot + geom_point() + geom_hline(yintercept = 0) + scale_x_discrete(guide = guide_axis(n.dodge=3))

#### Short Answer 8
​
<em>Prompt:</em> Describe the relationship between education level and the residuals in the graph above. What does the graph tell us about the presence and nature of heteroskedasticity in the regression model?

<font color="red">Answer here in red</font>

To test for heteroskedasticity formally, let's perform the White Test. First, store the residuals from the previous regression in ``census_data``.

<em>Tested Objects: ``census_data`` (checks to see that residuals were added properly).

In [ ]:
#Quiz 6

census_data <- mutate(census_data, resid = reg5$residuals)

head(census_data$resid, 10) #Displays the residuals in the dataframe

test_6() #test3

Next, generate a variable for the squared residuals, then run the required auxiliary regression.

<em>Tested Objects:</em> ``WT`` (the auxiliary regression).

In [ ]:
#Quiz 7
census_data <- census_data %>%
    mutate(residsq = ???) #Generate squared residuals

WT <- lm(??? ~ educ + sex + immstat + educ*educ + sex*sex + immstat*immstat + educ*sex + educ*immstat + sex*immstat, data = census_data)

summary(WT)

test_7() #test4

Last, calculate a test statistic from the regression above and perform the appropriate hypothesis test at the 5% significance level.

<em>Tested Objects:</em> ``BPstat``.

In [ ]:
#Quiz 8
BPstat <- ???*nrow(census_data) #Compute test statistic (what's the value?)

BPstat > qchisq(p = 0.05, df=???) #Compares test stat to critical value (what should df be)

#### Short Answer 9

<em>Prompt:</em> What is the null hypothesis that we are testing here? You can explain this in words or algebra. Do we reject the null hypothesis of the White Test? What does this mean for the regression model's heteroskedasticity?

<font color="red">Answer here in red</font>

Write R code below that adjusts the standard errors for heteroskedasticity. Compare these standard errors with those of the OLS regression.

<em>Tested Objects:</em> The robust standard errors will be graded manually.

In [ ]:
#Quiz 9

coeftest(???)
summary(reg5)

#### Short Answer 10

<em>Prompt:</em> What happened to the standard errors of the regression estimates when we accounted for heteroskedasticity? What does this imply?

<font color="red">Answer here in red</font>